In [1]:
import numpy as np
import pandas as pd

import re

from neo4j import GraphDatabase
from py2neo import Graph

todoremove: test code to create random nodes

In [2]:
# import random

# transaction_id = [i for i in range(1,50001)]
# vendor_number = np.random.randint(low=1, high=2500, size=(50000,))

# transaction_amount = np.random.randint(low=20, high=12500000, size=(50000))

# transaction_types = ['cash_withdrawal', 'cash_deposit', 'transfer_domestic', 'transfer_international']

# random_integers = [random.randint(0,3) for i in range(0,50000)]
# transaction_list = [transaction_types[i] for i in random_integers]

# transaction_data = {'transaction_id': transaction_id,
#                     'vendor_number': list(vendor_number),
#                    'transaction_amount': list(transaction_amount),
#                    'transaction_type': transaction_list,}

# transaction_DataFrame = pd.DataFrame(transaction_data)

In [3]:
# transaction_list = transaction_DataFrame.values.tolist()

# transaction_execution_commands = []

# for i in transaction_list:
#     neo4j_create_statement = 'create (t:Transaction {transaction_id:' + str(i[0]) + ', vendor_number: '+ str(i[1]) +'})'
#     transaction_execution_commands.append(neo4j_create_statement)

Connection to local Neo4j server

In [4]:
data_base_connection = GraphDatabase.driver(uri='bolt://localhost:7687', auth=('neo4j', 'test'))
session = data_base_connection.session()

Helper function to batch execute Cypher queries

In [5]:
def execute_transactions(transaction_execution_commands):
    data_base_connection = GraphDatabase.driver(uri='bolt://localhost:7687', auth=('neo4j', 'test'))
    session = data_base_connection.session()
    for i in transaction_execution_commands:
        session.run(i)

# execute_transactions(transaction_execution_commands)

Read nodes & edges sample files

In [6]:
fname_in_nodes = 'Nodes Lineage Sample.csv'
fname_in_edges = 'Edges Lineage Sample.csv'

in_nodes = pd.read_csv(fname_in_nodes)
in_edges = pd.read_csv(fname_in_edges)


pandas profiling EDA on nodes & edges

In [7]:
# from pandas_profiling import ProfileReport

# profile_nodes = ProfileReport(in_nodes, title="Nodes Report")
# profile_nodes

# profile_edges = ProfileReport(in_edges, title="Edges Report")
# profile_edges

todo: replace fillna - required for creating cypher code to build nodes

# Nodes

Fill na (only present in Description field)

In [8]:
df_nodes = in_nodes.drop_duplicates().fillna('na')

Automatically create code for node dataframes

In [9]:
# df = ([list(df_nodes['labels(n)'].unique()),list(df_nodes['labels(n)'].unique())])
df_nodes_code = pd.DataFrame([list(df_nodes['labels(n)'].unique()), ['df_nd_' + re.sub('[^0-9a-zA-Z]+', '', name).lower() for name in list(df_nodes['labels(n)'].unique())]]).T
df_nodes_code.columns=['label', 'df_name']
df_nodes_code['code'] = df_nodes_code['df_name'] + ' = df_nodes[df_nodes["labels(n)"]=="' + df_nodes_code['label'] + '"].reset_index(drop=True)'
print(df_nodes_code['code'])

0     df_nd_system = df_nodes[df_nodes["labels(n)"]=...
1     df_nd_datafeed = df_nodes[df_nodes["labels(n)"...
2     df_nd_systemdataconcept = df_nodes[df_nodes["l...
3     df_nd_systemdataelement = df_nodes[df_nodes["l...
4     df_nd_businessdataelement = df_nodes[df_nodes[...
5     df_nd_boundaryvalue = df_nodes[df_nodes["label...
6     df_nd_boundaryset = df_nodes[df_nodes["labels(...
7     df_nd_businessdataconcept = df_nodes[df_nodes[...
8     df_nd_boundarydimension = df_nodes[df_nodes["l...
9     df_nd_orgunit = df_nodes[df_nodes["labels(n)"]...
10    df_nd_materialtransformation = df_nodes[df_nod...
11    df_nd_businessdatadomain = df_nodes[df_nodes["...
12    df_nd_report = df_nodes[df_nodes["labels(n)"]=...
13    df_nd_regulator = df_nodes[df_nodes["labels(n)...
Name: code, dtype: object


Create subset for label types

In [10]:
df_nd_system = df_nodes[df_nodes['labels(n)']=='[System]'].reset_index(drop=True)
df_nd_datafeed = df_nodes[df_nodes['labels(n)']=='[Data Feed]']
df_nd_systemdataconcept = df_nodes[df_nodes['labels(n)']=='[System Data Concept]']
df_nd_systemdataelement = df_nodes[df_nodes['labels(n)']=='[System Data Element]']
df_nd_businessdataelement = df_nodes[df_nodes['labels(n)']=='[Business Data Element]']
df_nd_boundaryvalue = df_nodes[df_nodes['labels(n)']=='[Boundary Value]']
df_nd_boundaryset = df_nodes[df_nodes['labels(n)']=='[Boundary Set]']
df_nd_businessdataconcept = df_nodes[df_nodes['labels(n)']=='[Business Data Concept]']
df_nd_boundarydimension = df_nodes[df_nodes['labels(n)']=='[Boundary Dimension]']
df_nd_orgunit = df_nodes[df_nodes['labels(n)']=='[Org Unit]']
df_nd_materialtransformation = df_nodes[df_nodes['labels(n)']=='[Material Transformation]']
df_nd_businessdatadomain = df_nodes[df_nodes['labels(n)']=='[Business Data Domain]']
df_nd_report = df_nodes[df_nodes['labels(n)']=='[Report]']
df_nd_regulator = df_nodes[df_nodes['labels(n)']=='[Regulator]']

# Edges

In [ ]:
in_edges['type(r)'].unique()

In [ ]:
in_edges[in_edges['n1.Name'].str.contains('\[')]['n1.Name'].unique()

In [ ]:
graph = Graph("bolt://localhost:7687", user="neo4j", password="test")
tx = graph.begin()

# for index, row in df_nodes.iterrows():
#     tx.evaluate('''
#        MERGE (a:person1 {property:$Person1})
#        MERGE (b:person2 {property:$Person2})
#        MERGE (a)-[r:R_TYPE]->(b)
#        ''', parameters = {'Person1': int(row['n.Name']), 'Person2': int(row['Person2'])})

for index, row in df_nodes.iterrows():
    tx.evaluate('''
        MERGE (a:name {label:$label, description:$description})''',
        parameters = {'labels(n)': row['labels(n)'], 'name': row['n.Name'], 'label': row['n.Label'], 'description': row['n.Description']})

tx.commit()

In [ ]:
# stmt = 'LOAD CSV WITH HEADERS FROM '+ fname_in_nodes + ' AS line \
#         CREATE (:Artist {name: line.Name, year: toInteger(line.Year)})'

# session.run(stmt)